In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/Users/mansipandya/Desktop/Lung-Cancer-Prediction'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
from LungCancerPrediction.constants import *
from LungCancerPrediction.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        # params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        # self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [7]:
import os
from LungCancerPrediction import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [8]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up

    def transform(self,data):
        data = data.drop(columns=['index', 'Patient Id'], axis=1)
        data["Level"] = data["Level"].replace({'High': 2, 'Medium': 1, 'Low': 0})
        data = data.infer_objects(copy=False)
        return data

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        data = self.transform(data)
        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

In [19]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-06-03 19:19:26,968: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-03 19:19:26,974: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-06-03 19:19:26,975: INFO: common: created directory at: artifacts]
[2024-06-03 19:19:26,976: INFO: common: created directory at: artifacts/data_transformation]
[2024-06-03 19:19:26,994: INFO: 78788802: Splited data into training and test sets]
[2024-06-03 19:19:26,995: INFO: 78788802: (750, 24)]
[2024-06-03 19:19:26,996: INFO: 78788802: (250, 24)]
(750, 24)
(250, 24)


/var/folders/13/y7btbnkj3vz8xckjs7_qg8g40000gn/T/ipykernel_49250/78788802.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["Level"] = data["Level"].replace({'High': 2, 'Medium': 1, 'Low': 0})
